<a href="https://colab.research.google.com/github/minkyung10/class2022Spring/blob/main/huggingface_gradio%20(0525%20%EC%88%98%EC%97%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

### [Image classification](https://huggingface.co/tasks/image-classification)

image classification을 해주는 모델 중 하나를 선택해서 구현한다

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use

In [ ]:
!pip install transformers

In [3]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg' #그림 가져오는 것
image = Image.open(requests.get(url, stream=True).raw)  #이미지를 숫자 형태로, 데이터로 가져 온다

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

#feature extractor: 무수한 정보들 중에서 대상을 구분지을 수 있는  특징을 압축적으로 추출해낸다
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


Demo in Gradio

gradio 사용해서 Function으로 만든다

함수의 형태로 표현하기



In [4]:
def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

In [5]:
import os    #사진 다운로드하는 것
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

0

넣고 싶은 사진들 List화 해서 써넣기

In [6]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://41651.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fe4031e2a50>,
 'http://127.0.0.1:7860/',
 'https://41651.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [8]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

[{'score': 0.10731077939271927,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.0877445638179779,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338413640856743,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.046672213822603226,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095887809991837,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

Demo in Gradio

In [10]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)   #result는 list형식으로 5개가 제시된다
  df = pd.DataFrame(result)   #result - Dataframe 형식으로 바꾸지
  return df

In [11]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [12]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://21603.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fe2cd75a810>,
 'http://127.0.0.1:7861/',
 'https://21603.gradio.app')

### [Token classification](https://huggingface.co/tasks/token-classification)

NLP 중 하나의 분야

Natural Language에서 중요한 정보를 인식하는 것

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

#nlp가 모델 이름이 된다
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


Demo in Gradio

In [14]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df

In [15]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [16]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://47839.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fe2ca6f8850>,
 'http://127.0.0.1:7862/',
 'https://47839.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

주어진 문장이 있을 때 얼마나 유사한가를 측정

입력은 벡터(embeddings)로 바꾸어줘야 함


e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers

In [19]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

[[ 1.88456848e-01  1.74256176e-01  5.44777215e-02  2.90517420e-01
   1.67664081e-01 -4.72069457e-02  6.45580173e-01  1.59808800e-01
   2.26892605e-01 -3.08905598e-02  2.55883545e-01 -5.25876768e-02
  -2.26101175e-01 -5.71063384e-02  1.30426243e-01  1.24953501e-01
   3.17496210e-01  1.94444105e-01 -5.86325705e-01 -1.25861000e-02
   6.09909177e-01  1.64327413e-01  3.33113708e-02 -2.73830682e-01
  -2.89757580e-01 -2.11196810e-01 -2.26138867e-02 -1.70359373e-01
   1.61589950e-01  6.08273745e-02 -2.41624042e-01  1.85792103e-01
   4.27409589e-01  1.92951798e-01 -7.23447800e-02  1.66111022e-01
   1.04428008e-01  2.04772323e-01  2.11167201e-01  1.99740008e-01
  -9.40827355e-02 -1.73836619e-01  6.42733201e-02  2.80254930e-01
  -2.95305759e-01  6.20952919e-02  1.04276791e-01 -2.36439407e-02
   1.29131526e-01 -1.26174763e-01 -1.78990051e-01  3.70057598e-02
  -6.12506211e-01  5.02983071e-02  1.77303478e-01  2.24941239e-01
   1.73860714e-01 -3.84028591e-02 -2.12868378e-01  2.58492470e-01
  -1.21016

In [20]:
len(embeddings[1])

384

두 벡터(embeddings)를 사용해서 유사도를 구할 수 있음

이 유사도는 두 벡터 사이의 각도의 cos을 구하면 된다 

In [21]:
#두 벡터가 이루는 cos값을 계산해줌
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
cosine_scores

tensor([[0.5726]])

Demo in Gradio

In [22]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [23]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [24]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://29126.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fe2c145d250>,
 'http://127.0.0.1:7863/',
 'https://29126.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable
